In [0]:
import sys
import torch  
import gym
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
from scipy.optimize import least_squares

In [0]:
torch.cuda.is_available()

True

In [0]:
# Constants
GAMMA = 0.99

class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=3e-4):
        super(PolicyNetwork, self).__init__()

        self.num_actions = num_actions
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, num_actions)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.softmax(self.linear2(x), dim=1)
        return x 
    
    def get_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.forward(Variable(state))
        #Choose action with regard to policy
        highest_prob_action = np.random.choice(self.num_actions, p=np.squeeze(probs.detach().numpy()))
        log_prob = torch.log(probs.squeeze(0)[highest_prob_action]) #log for gradient
        return highest_prob_action, log_prob

In [0]:
class ValueNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, learning_rate=3e-4):
        super(ValueNetwork, self).__init__()

        self.num_actions = num_actions
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, 1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
    
    def forward(self, state):
        state = Variable(torch.from_numpy(state).float().unsqueeze(0))
        value = F.relu(self.linear1(state))
        value = self.linear2(value)
        return value

In [0]:
def update_policy(policy_network, trajectories_gradient):
    policy_network.optimizer.zero_grad()    
    policy_gradient = torch.stack(trajectories_gradient).sum()
    policy_gradient.backward()
    policy_network.optimizer.step()

In [0]:
def count_trajectory(rewards, log_probs, baseline):
    discounted_rewards = []

    for t in range(len(rewards)):
        Gt = 0 
        pw = 0
        for r in rewards[t:]:
            Gt = Gt + GAMMA**pw * r
            pw = pw + 1
        discounted_rewards.append(Gt)
        
    discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9) # normalize discounted rewards

    policy_gradient = []
    for log_prob, Gt, bs in zip(log_probs, discounted_rewards, baseline):
        if log_prob == 0:
            policy_gradient.append(torch.tensor([[0.0]], requires_grad = True))
        else:
            policy_gradient.append(-log_prob * (Gt - bs))
    
    policy_gradient = torch.stack(policy_gradient).sum()
    return policy_gradient

In [0]:
def align_columns_zeros(array):
    max_length = max(list(map(lambda x: len(x), array)))
    for col in range(len(array)):
        array[col] = np.pad(array[col], max_length - len(array[col]), 'constant', constant_values = 0)[max_length - len(array[col]):]
    return array

In [0]:
def baseline_approximation(policy_net, value_net, n_trajectories, n_epoch = 4000, early_stopping_rounds = 250):
    max_steps = 10000
    min_loss = float('inf')
    stopping_rounds = 0
    epoch = 1
    while(stopping_rounds < early_stopping_rounds and epoch < n_epoch):
        r_gradient = []
        rewards = [[] for i in range(n_trajectories)]
        log_probs = [[] for i in range(n_trajectories)]
        baseline_values = [[] for i in range(n_trajectories)]
    
        for trajectory in range(n_trajectories):
            state = env.reset()
            
            for steps in range(max_steps):
                baseline = value_net.forward(state)
                action, log_prob = policy_net.get_action(state)
                new_state, reward, done, _ = env.step(np.array(action))
                baseline_values[trajectory].append(baseline)
                log_probs[trajectory].append(log_prob)
                rewards[trajectory].append(reward)
                if done:                    
                    break
                state = new_state
        rewards = align_columns_zeros(rewards)
        log_probs = align_columns_zeros(log_probs)
        baseline_values = align_columns_zeros(baseline_values)
        for col in range(len(rewards)):
            traj = count_trajectory(rewards[col], log_probs[col], baseline_values[col])
            traj.backward()
            deriv = []
            for param in policy_net.parameters():
              deriv.append(param.grad.view(param.grad.numel()))
            deriv = torch.cat([tensor for tensor in deriv], 0)
            r_gradient.append(deriv)

        r_gradient = torch.stack(r_gradient)
        value_loss = torch.var(r_gradient, 1).sum()
        if value_loss < min_loss:
            min_loss = value_loss
            torch.save(value_net, 'model.pth')
            print('{}. Current minimum value loss = {}'.format(epoch, value_loss))
            stopping_rounds = 0
        else:
            stopping_rounds += 1
        epoch += 1
        value_net.optimizer.zero_grad()
        value_loss.backward()
        value_net.optimizer.step()
    value_net = torch.load('model.pth')

In [0]:
def cart_pole_baseline(value_net, n_trajectories = 2, episode_num = 1500, baseline_retrain = False, retrain_episodes = 50):
    env = gym.make('CartPole-v0')
    policy_net = PolicyNetwork(env.observation_space.shape[0], env.action_space.n, 256)
    

    max_episode_num = episode_num
    max_steps = 10000
    numsteps = [[] for i in range(n_trajectories)]
    avg_numsteps = [[] for i in range(n_trajectories)]
    all_rewards = [[] for i in range(n_trajectories)]

    for episode in range(1, max_episode_num + 1):
        r_gradient = []
        rewards = [[] for i in range(n_trajectories)]
        baseline_values = []
        
        for trajectory in range(n_trajectories):
            state = env.reset()
            log_probs = []
            
            for steps in range(max_steps):
                #env.render()
                baseline = value_net.forward(state)
                action, log_prob = policy_net.get_action(state)
                new_state, reward, done, _ = env.step(action)
                log_probs.append(log_prob)
                rewards[trajectory].append(reward)
                baseline_values.append(baseline)
                
                if done:
                    traj = count_trajectory(rewards[trajectory], log_probs, baseline_values)                       
                    r_gradient.append(traj)
                    numsteps[trajectory].append(steps)
                    avg_numsteps[trajectory].append(np.mean(numsteps[trajectory][-10:]))
                    all_rewards[trajectory].append(np.sum(rewards[trajectory]))
                    break

                state = new_state
                
        update_policy(policy_net, r_gradient)                 
        rewards = align_columns_zeros(rewards)
        #print(all_rewards)
        if episode % 10 == 0:
            sys.stdout.write("episode: {}, total mean reward among trajectories: {}, average_reward_among_trajectories: {}\n".\
                                     format(episode, np.round(np.mean(np.sum(rewards, axis = 1)), decimals = 3),\
                                            np.round(np.mean(np.mean(all_rewards, axis = 0)[-10:]), decimals = 3)))
            
        if baseline_retrain == True:
            if episode % retrain_episodes == 0:
                baseline_approximation(policy_net, value_net, 3, n_epoch = 1000, early_stopping_rounds=10)
        
    return all_rewards, avg_numsteps

In [0]:
%%time
env = gym.make('CartPole-v0')
policy_net = PolicyNetwork(env.observation_space.shape[0], env.action_space.n, 256)
value_net = ValueNetwork(env.observation_space.shape[0], env.action_space.n, 256)
baseline_approximation(policy_net, value_net, 15, early_stopping_rounds=300)

1. Current minimum value loss = 7.86085319519043


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type ValueNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


RuntimeError: ignored

In [0]:
%%time
statistics_all = []
statistics_mean = []
for game in tqdm(range(10)):
  all_rewards, mean_rewards = cart_pole(1, 2000)
  statistics_all.append(all_rewards)
  statistics_mean.append(mean_rewards)